In [1]:
%pylab inline
import pandas as pd
import numpy as np
import keras

Populating the interactive namespace from numpy and matplotlib


Using TensorFlow backend.
/home/said/anaconda3/envs/end2end/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


### Resnet & VGG16

In [1]:
from keras.models import load_model
res_mod = load_model('1152x896_s10_prt_addtopf2_1.h5', compile=False)
vgg_mod = load_model('1152x896_s10_prt_addvgg20.h5', compile=False)
hybrid_mod = load_model('1152x896_s10_prt_addvgg53.h5', compile=False)

Using TensorFlow backend.
/home/said/anaconda3/envs/end2end/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


OSError: Unable to open file (unable to open file: name = '1152x896_s10_prt_addtopf2_1.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

On your machine, you shall have folders like the following:

- ./full_test_1152x896
- ./full_test_1152x896/neg
- ./full_test_1152x896/pos

The "neg" being the negative images and "pos" being the positive images.

In [3]:
from dm_image import DMImageDataGenerator
test_imgen = DMImageDataGenerator(featurewise_center=True)
test_imgen.mean = 52.18
test_generator = test_imgen.flow_from_directory(
    './full_test_1152x896', target_size=(1152, 896), target_scale=None,
    rescale_factor=0.003891,
    equalize_hist=False, dup_3_channels=True, 
    classes=['neg', 'pos'], class_mode='categorical', batch_size=4, 
    shuffle=False)

Found 376 images belonging to 2 classes.


In [4]:
from dm_keras_ext import DMAucModelCheckpoint
res_auc, res_y_true, res_y_pred = DMAucModelCheckpoint.calc_test_auc(
    test_generator, res_mod, test_samples=test_generator.nb_sample, return_y_res=True)
print res_auc

[ 0.86367945  0.86367945]


In [5]:
from dm_keras_ext import DMAucModelCheckpoint
vgg_auc, vgg_y_true, vgg_y_pred = DMAucModelCheckpoint.calc_test_auc(
    test_generator, vgg_mod, test_samples=test_generator.nb_sample, return_y_res=True)
print vgg_auc

[ 0.82536263  0.82536263]


In [6]:
from dm_keras_ext import DMAucModelCheckpoint
hybrid_auc, hybrid_y_true, hybrid_y_pred = DMAucModelCheckpoint.calc_test_auc(
    test_generator, hybrid_mod, test_samples=test_generator.nb_sample, return_y_res=True)
print hybrid_auc

[ 0.84914548  0.84914548]


In [7]:
print np.array_equal(res_y_true, vgg_y_true)
print np.array_equal(res_y_true, hybrid_y_true)

True
True


In [9]:
from sklearn.metrics import roc_auc_score
all_mod_y_pred_avg = (res_y_pred[:,1] + vgg_y_pred[:,1] + hybrid_y_pred[:,1])/3
print roc_auc_score(res_y_true[:,1], all_mod_y_pred_avg)

0.88507827086


In [10]:
from dm_keras_ext import DMAucModelCheckpoint
res_auc_aug, res_y_true_aug, res_y_pred_aug = DMAucModelCheckpoint.calc_test_auc(
    test_generator, res_mod, test_samples=test_generator.nb_sample, return_y_res=True, test_augment=True)
print res_auc_aug

[ 0.87832831  0.87832831]


In [11]:
from dm_keras_ext import DMAucModelCheckpoint
vgg_auc_aug, vgg_y_true_aug, vgg_y_pred_aug = DMAucModelCheckpoint.calc_test_auc(
    test_generator, vgg_mod, test_samples=test_generator.nb_sample, return_y_res=True, test_augment=True)
print vgg_auc_aug

[ 0.85919862  0.85919862]


In [12]:
from dm_keras_ext import DMAucModelCheckpoint
hybrid_auc_aug, hybrid_y_true_aug, hybrid_y_pred_aug = DMAucModelCheckpoint.calc_test_auc(
    test_generator, hybrid_mod, test_samples=test_generator.nb_sample, return_y_res=True, test_augment=True)
print hybrid_auc_aug

[ 0.88237829  0.88237829]


In [13]:
from sklearn.metrics import roc_auc_score
all_mod_y_pred_avg_aug = (res_y_pred_aug[:,1] + vgg_y_pred_aug[:,1] + hybrid_y_pred_aug[:,1])/3
print roc_auc_score(res_y_true_aug[:,1], all_mod_y_pred_avg_aug)

0.907827086026


Done.